In [1]:
import numpy as np
import random
import pdb
#from Bio import SeqIO
from sklearn import preprocessing
import urllib3


window_size=11
win_it_size = 50
inOneHotVectSize=22 #
outOneHotVectSize=4

http        = urllib3.PoolManager()
response    = http.request('GET', "http://calla.rnet.missouri.edu/cheng_courses/mlbioinfo/ss_train.txt")
data        = response.data.decode('utf-8')
splitByLine = data.split('\n')


response2   = http.request('GET', "http://calla.rnet.missouri.edu/cheng_courses/mlbioinfo/ss_test.txt")
dataTest    = response.data.decode('utf-8')
splitByLine2 = dataTest.split('\n')

NUM_TRAIN_SEQ = 1180
NUM_TEST_SEQ  = 126

#preprocessing
#label Encode

outLab        = ['C','H','E','z']   #z is edge
outLabEncoder = preprocessing.LabelEncoder()
outLabEncoder.fit(outLab)

inLab        = ['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','X','Y','V','z'] #z is edge
inLabEncoder = preprocessing.LabelEncoder()
inLabEncoder.fit(inLab) 

encTrainArray = []
for i in range(0, NUM_TRAIN_SEQ):
	arrayOfCharsIn  = list(splitByLine[(i*4)+2])
	arrayOfCharsOut = list(splitByLine[(i*4)+3])
	encodedIn = inLabEncoder.transform(arrayOfCharsIn)
	encodedOut = outLabEncoder.transform(arrayOfCharsOut)
	encTrainArray.append([encodedIn, encodedOut])


#Testing
encTestArray = []
for i in range(0, NUM_TEST_SEQ):
	arrayOfCharsInTest  = list(splitByLine2[(i*4)+2])
	arrayOfCharsOutTest = list(splitByLine2[(i*4)+3])
	encodedInTest = inLabEncoder.transform(arrayOfCharsInTest)
	encodedOutTest = outLabEncoder.transform(arrayOfCharsOutTest)
	encTestArray.append([encodedInTest, encodedOutTest]) 

#OneHot
#traingin
inLab    = np.array(inLab)
outLab   = np.array(outLab)
inLab    = inLabEncoder.transform(inLab)
outLab   = outLabEncoder.transform(outLab)

inOneHot  = preprocessing.OneHotEncoder()
inOneHot.fit(inLab.reshape(-1,1))
outOneHot = preprocessing.OneHotEncoder()
outOneHot.fit(outLab.reshape(-1,1))

x_train = np.ones((1, window_size, inOneHotVectSize))
y_train = np.ones((1, window_size, outOneHotVectSize))

inEnds  = np.zeros(22)
outEnds = np.zeros(4)


x_test  = np.ones((1, window_size, inOneHotVectSize))
y_test  = np.ones((1, window_size, outOneHotVectSize))

encodingForInBlank  = np.zeros((1,inOneHotVectSize))
encodingForOutBlank = np.zeros((1, outOneHotVectSize))
encodingForInBlank[0][inOneHotVectSize-1]   = 1
encodingForOutBlank[0][outOneHotVectSize-1] = 1

#Assemble Training Set
for i in range(0, NUM_TRAIN_SEQ):
	xPiece = inOneHot.transform(np.array(encTrainArray[i][0]).reshape(-1,1)).toarray()
	yPiece = outOneHot.transform(np.array(encTrainArray[i][1]).reshape(-1,1)).toarray()
	print(xPiece.shape)
	start = random.randint(0, window_size-1)
	for jjj in range(-1*start, xPiece.shape[0], win_it_size):
		print("i",i,"j",jjj)
		#Left edges
		if jjj<0:
			smallXPiece = xPiece[0:window_size+jjj]
			smallYPiece = yPiece[0:window_size+jjj]
			borderNeeded = abs(jjj)
			for k in range(0, borderNeeded):
				smallXPiece = np.vstack((encodingForInBlank, smallXPiece))
				smallYPiece = np.vstack((encodingForOutBlank, smallYPiece))
			x_train = np.vstack((x_train, np.expand_dims(smallXPiece, 0)))
			y_train = np.vstack((y_train, np.expand_dims(smallYPiece, 0)))

		#Right edges
		elif jjj>xPiece.shape[0]-window_size:
			smallXPiece   = xPiece[jjj:]
			smallYPiece   = yPiece[jjj:]
			borderNeeded  = window_size - smallXPiece.shape[0]
			for k in range(0, borderNeeded):
				smallXPiece   = np.vstack((smallXPiece, encodingForInBlank))
				smallYPiece   = np.vstack((smallYPiece, encodingForOutBlank))

			x_train = np.vstack((x_train, np.expand_dims(smallXPiece, 0)))
			y_train = np.vstack((y_train, np.expand_dims(smallYPiece, 0)))
		else:
			smallXPiece = xPiece[jjj:jjj+window_size]
			smallYPiece = yPiece[jjj:jjj+window_size]
			x_train = np.vstack((x_train, np.expand_dims(smallXPiece, 0)))
			y_train = np.vstack((y_train, np.expand_dims(smallYPiece, 0)))


#Assemble Testing Set

for i in range(0, NUM_TEST_SEQ):
	xPiece = inOneHot.transform(np.array(encTestArray[i][0]).reshape(-1,1)).toarray()
	yPiece = outOneHot.transform(np.array(encTestArray[i][1]).reshape(-1,1)).toarray()
	print(xPiece.shape)
	for jjj in range(-1*int(window_size/2), xPiece.shape[0]+int(window_size/2)):
		print("i",i,"j",jjj)
		#Left edges
		if jjj<0:
			smallXPiece = xPiece[0:window_size+jjj]
			smallYPiece = yPiece[0:window_size+jjj]
			borderNeeded = abs(jjj)
			for k in range(0, borderNeeded):
				smallXPiece = np.vstack((encodingForInBlank, smallXPiece))
				smallYPiece = np.vstack((encodingForOutBlank, smallYPiece))
			x_test = np.vstack((x_test, np.expand_dims(smallXPiece, 0)))
			y_test = np.vstack((y_test, np.expand_dims(smallYPiece, 0)))

		#Right edges
		elif jjj>xPiece.shape[0]-window_size:
			smallXPiece   = xPiece[jjj:]
			smallYPiece   = yPiece[jjj:]
			borderNeeded  = window_size - smallXPiece.shape[0]
			for k in range(0, borderNeeded):
				smallXPiece   = np.vstack((smallXPiece, encodingForInBlank))
				smallYPiece   = np.vstack((smallYPiece, encodingForOutBlank))

			x_test = np.vstack((x_test, np.expand_dims(smallXPiece, 0)))
			y_test = np.vstack((y_test, np.expand_dims(smallYPiece, 0)))
		else:
			smallXPiece = xPiece[jjj:jjj+window_size]
			smallYPiece = yPiece[jjj:jjj+window_size]
			x_test = np.vstack((x_test, np.expand_dims(smallXPiece, 0)))
			y_test = np.vstack((y_test, np.expand_dims(smallYPiece, 0)))




#to eliminate the all 1 initialization row needed for shape preservation
x_train = np.delete(x_train, 0,0)
y_train = np.delete(y_train, 0,0)
x_test  = np.delete(x_test, 0,0)
y_test  = np.delete(y_test, 0,0)

(162, 22)
i 0 j -4
i 0 j 46
i 0 j 96
i 0 j 146
(327, 22)
i 1 j -5
i 1 j 45
i 1 j 95
i 1 j 145
i 1 j 195
i 1 j 245
i 1 j 295
(185, 22)
i 2 j -6
i 2 j 44
i 2 j 94
i 2 j 144
(415, 22)
i 3 j -8
i 3 j 42
i 3 j 92
i 3 j 142
i 3 j 192
i 3 j 242
i 3 j 292
i 3 j 342
i 3 j 392
(141, 22)
i 4 j -4
i 4 j 46
i 4 j 96
(279, 22)
i 5 j -4
i 5 j 46
i 5 j 96
i 5 j 146
i 5 j 196
i 5 j 246
(437, 22)
i 6 j 0
i 6 j 50
i 6 j 100
i 6 j 150
i 6 j 200
i 6 j 250
i 6 j 300
i 6 j 350
i 6 j 400
(201, 22)
i 7 j -1
i 7 j 49
i 7 j 99
i 7 j 149
i 7 j 199
(271, 22)
i 8 j -7
i 8 j 43
i 8 j 93
i 8 j 143
i 8 j 193
i 8 j 243
(413, 22)
i 9 j 0
i 9 j 50
i 9 j 100
i 9 j 150
i 9 j 200
i 9 j 250
i 9 j 300
i 9 j 350
i 9 j 400
(401, 22)
i 10 j -9
i 10 j 41
i 10 j 91
i 10 j 141
i 10 j 191
i 10 j 241
i 10 j 291
i 10 j 341
i 10 j 391
(159, 22)
i 11 j -4
i 11 j 46
i 11 j 96
i 11 j 146
(85, 22)
i 12 j -1
i 12 j 49
(106, 22)
i 13 j -1
i 13 j 49
i 13 j 99
(351, 22)
i 14 j -4
i 14 j 46
i 14 j 96
i 14 j 146
i 14 j 196
i 14 j 246
i 14 j 296


i 135 j 441
i 135 j 491
i 135 j 541
(247, 22)
i 136 j -7
i 136 j 43
i 136 j 93
i 136 j 143
i 136 j 193
i 136 j 243
(175, 22)
i 137 j -7
i 137 j 43
i 137 j 93
i 137 j 143
(143, 22)
i 138 j -2
i 138 j 48
i 138 j 98
(60, 22)
i 139 j -10
i 139 j 40
(228, 22)
i 140 j 0
i 140 j 50
i 140 j 100
i 140 j 150
i 140 j 200
(456, 22)
i 141 j 0
i 141 j 50
i 141 j 100
i 141 j 150
i 141 j 200
i 141 j 250
i 141 j 300
i 141 j 350
i 141 j 400
i 141 j 450
(605, 22)
i 142 j -4
i 142 j 46
i 142 j 96
i 142 j 146
i 142 j 196
i 142 j 246
i 142 j 296
i 142 j 346
i 142 j 396
i 142 j 446
i 142 j 496
i 142 j 546
i 142 j 596
(201, 22)
i 143 j -2
i 143 j 48
i 143 j 98
i 143 j 148
i 143 j 198
(552, 22)
i 144 j 0
i 144 j 50
i 144 j 100
i 144 j 150
i 144 j 200
i 144 j 250
i 144 j 300
i 144 j 350
i 144 j 400
i 144 j 450
i 144 j 500
i 144 j 550
(261, 22)
i 145 j -3
i 145 j 47
i 145 j 97
i 145 j 147
i 145 j 197
i 145 j 247
(341, 22)
i 146 j 0
i 146 j 50
i 146 j 100
i 146 j 150
i 146 j 200
i 146 j 250
i 146 j 300
(161, 22)


i 257 j 394
i 257 j 444
i 257 j 494
(126, 22)
i 258 j -8
i 258 j 42
i 258 j 92
(101, 22)
i 259 j -6
i 259 j 44
i 259 j 94
(356, 22)
i 260 j -9
i 260 j 41
i 260 j 91
i 260 j 141
i 260 j 191
i 260 j 241
i 260 j 291
i 260 j 341
(323, 22)
i 261 j -6
i 261 j 44
i 261 j 94
i 261 j 144
i 261 j 194
i 261 j 244
i 261 j 294
(349, 22)
i 262 j -6
i 262 j 44
i 262 j 94
i 262 j 144
i 262 j 194
i 262 j 244
i 262 j 294
i 262 j 344
(153, 22)
i 263 j -7
i 263 j 43
i 263 j 93
i 263 j 143
(76, 22)
i 264 j -4
i 264 j 46
(158, 22)
i 265 j -2
i 265 j 48
i 265 j 98
i 265 j 148
(124, 22)
i 266 j -5
i 266 j 45
i 266 j 95
(309, 22)
i 267 j -1
i 267 j 49
i 267 j 99
i 267 j 149
i 267 j 199
i 267 j 249
i 267 j 299
(449, 22)
i 268 j -4
i 268 j 46
i 268 j 96
i 268 j 146
i 268 j 196
i 268 j 246
i 268 j 296
i 268 j 346
i 268 j 396
i 268 j 446
(177, 22)
i 269 j 0
i 269 j 50
i 269 j 100
i 269 j 150
(376, 22)
i 270 j -5
i 270 j 45
i 270 j 95
i 270 j 145
i 270 j 195
i 270 j 245
i 270 j 295
i 270 j 345
(45, 22)
i 271 j -3
(

i 394 j 100
i 394 j 150
(237, 22)
i 395 j -4
i 395 j 46
i 395 j 96
i 395 j 146
i 395 j 196
(30, 22)
i 396 j -2
(501, 22)
i 397 j -5
i 397 j 45
i 397 j 95
i 397 j 145
i 397 j 195
i 397 j 245
i 397 j 295
i 397 j 345
i 397 j 395
i 397 j 445
i 397 j 495
(269, 22)
i 398 j -6
i 398 j 44
i 398 j 94
i 398 j 144
i 398 j 194
i 398 j 244
(172, 22)
i 399 j -2
i 399 j 48
i 399 j 98
i 399 j 148
(298, 22)
i 400 j -1
i 400 j 49
i 400 j 99
i 400 j 149
i 400 j 199
i 400 j 249
(129, 22)
i 401 j -10
i 401 j 40
i 401 j 90
(37, 22)
i 402 j -10
(58, 22)
i 403 j 0
i 403 j 50
(293, 22)
i 404 j -3
i 404 j 47
i 404 j 97
i 404 j 147
i 404 j 197
i 404 j 247
(68, 22)
i 405 j -2
i 405 j 48
(538, 22)
i 406 j -6
i 406 j 44
i 406 j 94
i 406 j 144
i 406 j 194
i 406 j 244
i 406 j 294
i 406 j 344
i 406 j 394
i 406 j 444
i 406 j 494
(294, 22)
i 407 j -2
i 407 j 48
i 407 j 98
i 407 j 148
i 407 j 198
i 407 j 248
(190, 22)
i 408 j -4
i 408 j 46
i 408 j 96
i 408 j 146
(173, 22)
i 409 j -6
i 409 j 44
i 409 j 94
i 409 j 144
(316

i 526 j -10
i 526 j 40
i 526 j 90
i 526 j 140
i 526 j 190
i 526 j 240
i 526 j 290
i 526 j 340
i 526 j 390
(210, 22)
i 527 j -7
i 527 j 43
i 527 j 93
i 527 j 143
i 527 j 193
(639, 22)
i 528 j -1
i 528 j 49
i 528 j 99
i 528 j 149
i 528 j 199
i 528 j 249
i 528 j 299
i 528 j 349
i 528 j 399
i 528 j 449
i 528 j 499
i 528 j 549
i 528 j 599
(339, 22)
i 529 j -8
i 529 j 42
i 529 j 92
i 529 j 142
i 529 j 192
i 529 j 242
i 529 j 292
(58, 22)
i 530 j -4
i 530 j 46
(351, 22)
i 531 j -7
i 531 j 43
i 531 j 93
i 531 j 143
i 531 j 193
i 531 j 243
i 531 j 293
i 531 j 343
(54, 22)
i 532 j -4
i 532 j 46
(218, 22)
i 533 j -7
i 533 j 43
i 533 j 93
i 533 j 143
i 533 j 193
(587, 22)
i 534 j -9
i 534 j 41
i 534 j 91
i 534 j 141
i 534 j 191
i 534 j 241
i 534 j 291
i 534 j 341
i 534 j 391
i 534 j 441
i 534 j 491
i 534 j 541
(158, 22)
i 535 j -2
i 535 j 48
i 535 j 98
i 535 j 148
(151, 22)
i 536 j 0
i 536 j 50
i 536 j 100
i 536 j 150
(314, 22)
i 537 j -9
i 537 j 41
i 537 j 91
i 537 j 141
i 537 j 191
i 537 j 241
i

...

i 115 j 116
i 115 j 117
i 115 j 118
i 115 j 119
i 115 j 120
i 115 j 121
i 115 j 122
i 115 j 123
i 115 j 124
i 115 j 125
i 115 j 126
i 115 j 127
i 115 j 128
i 115 j 129
i 115 j 130
i 115 j 131
i 115 j 132
i 115 j 133
i 115 j 134
i 115 j 135
i 115 j 136
i 115 j 137
i 115 j 138
i 115 j 139
i 115 j 140
i 115 j 141
i 115 j 142
i 115 j 143
i 115 j 144
i 115 j 145
i 115 j 146
i 115 j 147
i 115 j 148
i 115 j 149
i 115 j 150
i 115 j 151
i 115 j 152
i 115 j 153
i 115 j 154
i 115 j 155
i 115 j 156
i 115 j 157
i 115 j 158
i 115 j 159
i 115 j 160
i 115 j 161
i 115 j 162
i 115 j 163
i 115 j 164
i 115 j 165
i 115 j 166
i 115 j 167
i 115 j 168
i 115 j 169
i 115 j 170
i 115 j 171
i 115 j 172
i 115 j 173
i 115 j 174
i 115 j 175
i 115 j 176
i 115 j 177
i 115 j 178
i 115 j 179
i 115 j 180
i 115 j 181
i 115 j 182
i 115 j 183
i 115 j 184
i 115 j 185
i 115 j 186
i 115 j 187
i 115 j 188
i 115 j 189
i 115 j 190
i 115 j 191
i 115 j 192
i 115 j 193
i 115 j 194
i 115 j 195
i 115 j 196
i 115 j 197
i 115 j 198
i 11

i 117 j 123
i 117 j 124
i 117 j 125
i 117 j 126
i 117 j 127
i 117 j 128
i 117 j 129
i 117 j 130
i 117 j 131
i 117 j 132
i 117 j 133
i 117 j 134
i 117 j 135
i 117 j 136
i 117 j 137
i 117 j 138
i 117 j 139
i 117 j 140
i 117 j 141
i 117 j 142
i 117 j 143
i 117 j 144
i 117 j 145
i 117 j 146
i 117 j 147
i 117 j 148
i 117 j 149
i 117 j 150
i 117 j 151
i 117 j 152
i 117 j 153
i 117 j 154
i 117 j 155
i 117 j 156
i 117 j 157
i 117 j 158
i 117 j 159
i 117 j 160
i 117 j 161
i 117 j 162
i 117 j 163
i 117 j 164
i 117 j 165
i 117 j 166
i 117 j 167
i 117 j 168
i 117 j 169
i 117 j 170
i 117 j 171
i 117 j 172
i 117 j 173
i 117 j 174
i 117 j 175
i 117 j 176
i 117 j 177
i 117 j 178
i 117 j 179
i 117 j 180
i 117 j 181
i 117 j 182
i 117 j 183
i 117 j 184
i 117 j 185
i 117 j 186
i 117 j 187
i 117 j 188
i 117 j 189
i 117 j 190
i 117 j 191
i 117 j 192
i 117 j 193
i 117 j 194
i 117 j 195
i 117 j 196
i 117 j 197
i 117 j 198
i 117 j 199
i 117 j 200
i 117 j 201
i 117 j 202
i 117 j 203
i 117 j 204
i 117 j 205
i 11

i 120 j 15
i 120 j 16
i 120 j 17
i 120 j 18
i 120 j 19
i 120 j 20
i 120 j 21
i 120 j 22
i 120 j 23
i 120 j 24
i 120 j 25
i 120 j 26
i 120 j 27
i 120 j 28
i 120 j 29
i 120 j 30
i 120 j 31
i 120 j 32
i 120 j 33
i 120 j 34
i 120 j 35
i 120 j 36
i 120 j 37
i 120 j 38
i 120 j 39
i 120 j 40
i 120 j 41
i 120 j 42
i 120 j 43
i 120 j 44
i 120 j 45
i 120 j 46
i 120 j 47
i 120 j 48
i 120 j 49
i 120 j 50
i 120 j 51
i 120 j 52
i 120 j 53
i 120 j 54
i 120 j 55
i 120 j 56
i 120 j 57
i 120 j 58
i 120 j 59
i 120 j 60
i 120 j 61
i 120 j 62
i 120 j 63
i 120 j 64
i 120 j 65
i 120 j 66
i 120 j 67
i 120 j 68
i 120 j 69
i 120 j 70
i 120 j 71
i 120 j 72
i 120 j 73
i 120 j 74
i 120 j 75
i 120 j 76
i 120 j 77
i 120 j 78
i 120 j 79
i 120 j 80
i 120 j 81
i 120 j 82
i 120 j 83
i 120 j 84
i 120 j 85
i 120 j 86
i 120 j 87
i 120 j 88
i 120 j 89
i 120 j 90
i 120 j 91
i 120 j 92
i 120 j 93
i 120 j 94
i 120 j 95
i 120 j 96
i 120 j 97
i 120 j 98
i 120 j 99
i 120 j 100
i 120 j 101
i 120 j 102
i 120 j 103
i 120 j 104
i 120

i 122 j 249
i 122 j 250
i 122 j 251
i 122 j 252
i 122 j 253
i 122 j 254
i 122 j 255
i 122 j 256
i 122 j 257
i 122 j 258
i 122 j 259
i 122 j 260
i 122 j 261
i 122 j 262
i 122 j 263
i 122 j 264
i 122 j 265
i 122 j 266
i 122 j 267
i 122 j 268
i 122 j 269
i 122 j 270
i 122 j 271
i 122 j 272
(237, 22)
i 123 j -5
i 123 j -4
i 123 j -3
i 123 j -2
i 123 j -1
i 123 j 0
i 123 j 1
i 123 j 2
i 123 j 3
i 123 j 4
i 123 j 5
i 123 j 6
i 123 j 7
i 123 j 8
i 123 j 9
i 123 j 10
i 123 j 11
i 123 j 12
i 123 j 13
i 123 j 14
i 123 j 15
i 123 j 16
i 123 j 17
i 123 j 18
i 123 j 19
i 123 j 20
i 123 j 21
i 123 j 22
i 123 j 23
i 123 j 24
i 123 j 25
i 123 j 26
i 123 j 27
i 123 j 28
i 123 j 29
i 123 j 30
i 123 j 31
i 123 j 32
i 123 j 33
i 123 j 34
i 123 j 35
i 123 j 36
i 123 j 37
i 123 j 38
i 123 j 39
i 123 j 40
i 123 j 41
i 123 j 42
i 123 j 43
i 123 j 44
i 123 j 45
i 123 j 46
i 123 j 47
i 123 j 48
i 123 j 49
i 123 j 50
i 123 j 51
i 123 j 52
i 123 j 53
i 123 j 54
i 123 j 55
i 123 j 56
i 123 j 57
i 123 j 58
i 123 j 

i 124 j 426
i 124 j 427
i 124 j 428
i 124 j 429
i 124 j 430
i 124 j 431
i 124 j 432
i 124 j 433
i 124 j 434
i 124 j 435
i 124 j 436
i 124 j 437
i 124 j 438
i 124 j 439
i 124 j 440
i 124 j 441
i 124 j 442
i 124 j 443
i 124 j 444
i 124 j 445
i 124 j 446
i 124 j 447
i 124 j 448
i 124 j 449
i 124 j 450
i 124 j 451
i 124 j 452
i 124 j 453
(160, 22)
i 125 j -5
i 125 j -4
i 125 j -3
i 125 j -2
i 125 j -1
i 125 j 0
i 125 j 1
i 125 j 2
i 125 j 3
i 125 j 4
i 125 j 5
i 125 j 6
i 125 j 7
i 125 j 8
i 125 j 9
i 125 j 10
i 125 j 11
i 125 j 12
i 125 j 13
i 125 j 14
i 125 j 15
i 125 j 16
i 125 j 17
i 125 j 18
i 125 j 19
i 125 j 20
i 125 j 21
i 125 j 22
i 125 j 23
i 125 j 24
i 125 j 25
i 125 j 26
i 125 j 27
i 125 j 28
i 125 j 29
i 125 j 30
i 125 j 31
i 125 j 32
i 125 j 33
i 125 j 34
i 125 j 35
i 125 j 36
i 125 j 37
i 125 j 38
i 125 j 39
i 125 j 40
i 125 j 41
i 125 j 42
i 125 j 43
i 125 j 44
i 125 j 45
i 125 j 46
i 125 j 47
i 125 j 48
i 125 j 49
i 125 j 50
i 125 j 51
i 125 j 52
i 125 j 53
i 125 j 54
i 12

In [ ]:
print(x_train[1])
print(y_train[1])

In [2]:
from keras.models import Sequential
from keras.layers import *
from keras import optimizers, callbacks
#simple Keras model

#loss = 'categorical_hinge' # ok
loss = 'categorical_crossentropy' # best standart
#loss = 'mean_absolute_error' # bad
#loss = 'mean_squared_logarithmic_error' # new best (a little better


##TODO TODO TODO
## This is where we can insert our model,  it currently takes as input a onehot encoded vector of inputs restricted to a certain window size and outputs predictions for the 

#TASKS:
## NN architecture
## HyperParameter Tuning
## Plumming for splitting inputs during evalutaion
## Dealing with edge cases
## Incorporation of Homologous Sequences 
##TODO TODO TODO

##Can run now

#model = Sequential([Dense(outOneHotVectSize, input_shape=(window_size, inOneHotVectSize,)), Activation('relu')])

model=Sequential()
#model.add(Dense(outOneHotVectSize, input_shape=(window_size, inOneHotVectSize,)), Activation('relu')])
# model.add(Conv1D(filters, kernel_size, strides=1, padding='valid', input_shape=(window_size, inOneHotVectSize), activation="relu"))
# model.add(MaxPool1D(pool_size=3, strides=1, padding="valid"))
# model.add(Flatten())
# model.add(Dense(21, activation='relu'))
# model.add(Dense(3, activation='softmax'))

model.add(Conv1D(128, 11, strides=1, padding='same', input_shape=(window_size, inOneHotVectSize), activation="relu"))
model.add(Dropout(0.4))
model.add(Conv1D(64, 11, strides=1, padding='same', activation="relu"))
model.add(Dropout(0.4))
model.add(Conv1D(4, 11, strides=1, padding='same', activation="softmax"))

m = Sequential()
m.add(Conv1D(128, 5, padding='same', activation='relu', input_shape=(window_size, inOneHotVectSize)))
m.add(BatchNormalization())
m.add(Dropout(0.4))
m.add(Conv1D(128, 3, padding='same', activation='relu'))
m.add(BatchNormalization())
m.add(Dropout(0.4))
m.add(Conv1D(64, 3, padding='same', activation='relu'))
m.add(BatchNormalization())
m.add(Dropout(0.4))
#m.add(Flatten())
m.add(Dense(128, activation='relu'))
m.add(Dense(32, activation='relu'))
m.add(Dense(4, activation = 'softmax'))
opt = optimizers.Adam(lr=0.0005)
m.compile(optimizer=opt,
              loss=loss,
              metrics=['accuracy', 'mae'])
print(m.summary())
m.fit(x_train, y_train, epochs=100, batch_size=32)

print(model.summary())

# model.compile(loss='mse',
# 		optimizer='sgd',
# 		metrics=['accuracy'])
model.compile(optimizer=opt,loss=loss,metrics=['accuracy', 'mae'])
#x,y are numpy array
#model.fit(x_train, y_train, epochs=100, batch_size=1)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 11, 128)           14208     
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 128)           512       
_________________________________________________________________
dropout_3 (Dropout)          (None, 11, 128)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 11, 128)           49280     
_________________________________________________________________
batch_normalization_2 (Batch (None, 11, 128)           512       
_________________________________________________________________
dropout_4 (Dropout)          (None, 11, 128)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 11, 64)            24640     
__________

6343/6343 [==============================] - 13s 2ms/step - loss: 0.6130 - acc: 0.7327 - mean_absolute_error: 0.1795
Epoch 50/100
6343/6343 [==============================] - 12s 2ms/step - loss: 0.6148 - acc: 0.7314 - mean_absolute_error: 0.1807
Epoch 51/100
6343/6343 [==============================] - 14s 2ms/step - loss: 0.6104 - acc: 0.7349 - mean_absolute_error: 0.1793
Epoch 52/100
6343/6343 [==============================] - 14s 2ms/step - loss: 0.6135 - acc: 0.7341 - mean_absolute_error: 0.1794
Epoch 53/100
6343/6343 [==============================] - 13s 2ms/step - loss: 0.6057 - acc: 0.7396 - mean_absolute_error: 0.1778: 4s - loss: 0.6021 - a - ETA: 2s - loss: 0.6060 - acc
Epoch 54/100
6343/6343 [==============================] - 13s 2ms/step - loss: 0.6070 - acc: 0.7366 - mean_absolute_error: 0.1781
Epoch 55/100
6343/6343 [==============================] - 14s 2ms/step - loss: 0.6023 - acc: 0.7395 - mean_absolute_error: 0.1767
Epoch 56/100
6343/6343 [=========================

Non-trainable params: 0
_________________________________________________________________
None


In [4]:
#Evaluate model
#test_prediction = model.predict(x_test, batch_size=32)
test_prediction = m.predict(x_test, batch_size=32)
numCorrect = 0
numWrong   = 0
for ii,jj in zip(test_prediction, y_train):
	a = np.argmax(ii[int(window_size/2)])
	b = np.argmax(jj[int(window_size/2)])
	print(a,b)
	if(a==b):
		numCorrect = numCorrect +1
	else:
		numWrong  = numWrong+1
	


print(numWrong)
print(numCorrect)
print(numCorrect/(numCorrect+numWrong))

0 0
0 0
2 2
2 2
2 0
2 0
2 0
0 2
0 0
0 1
2 2
0 3
2 2
1 0
1 0
1 3
2 2
0 2
0 2
0 2
0 0
0 1
0 2
0 2
1 0
1 0
1 2
0 1
1 2
2 2
2 2
2 0
0 2
0 0
0 0
0 0
0 0
1 2
1 0
1 0
2 2
2 2
2 1
2 1
2 0
2 2
2 3
2 3
2 0
2 2
0 1
0 0
0 0
0 1
0 1
0 2
1 1
1 0
0 0
2 1
2 1
2 1
2 3
2 0
2 0
0 1
0 2
0 0
0 1
2 1
2 1
2 0
2 2
2 2
2 0
2 0
2 2
2 0
2 0
0 0
0 0
0 2
0 2
0 2
0 1
2 0
2 1
2 3
0 3
0 2
2 0
2 1
2 2
2 0
2 3
2 0
2 1
2 3
2 1
2 0
2 1
2 1
2 2
2 0
2 2
0 2
0 3
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
2 0
2 0
2 0
2 2
2 3
2 3
2 2
2 1
2 3
2 3
2 2
2 2
2 1
2 3
2 1
2 1
2 3
0 1
0 0
0 0
0 3
0 1
0 0
0 1
0 2
0 2
0 0
0 0
0 0
0 3
2 0
2 2
2 0
2 2
2 0
2 0
2 0
2 0
2 0
0 0
0 0
0 1
0 2
0 0
0 2
0 0
0 0
3 0
3 2
3 0
3 2
3 0
3 0
3 3
3 0
3 0
3 1
0 2
2 2
2 0
2 3
2 3
2 1
2 2
2 1
2 2
1 2
1 0
1 2
1 2
2 0
2 0
2 0
2 1
2 2
2 2
2 2
2 3
2 3
2 2
2 0
1 1
1 0
1 2
1 2
1 0
0 0
0 2
2 0
2 2
2 0
1 3
0 1
0 2
0 2
0 2
0 0
0 3
0 0
0 0
0 2
0 2
0 0
2 0
2 1
2 2
2 2
0 0
1 1
1 0
1 1
1 0
1 0
1 0
1 0
0 0
0 2
0 0
0 2
0 2
1 3
1 1
1 1
1 1
1 0
2 1
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
0 0


1 2
1 3
0 0
0 2
0 0
0 2
0 2
2 2
2 2
2 2
2 0
2 1
2 0
2 2
2 0
2 2
1 0
2 0
2 0
2 2
2 2
2 1
2 0
1 0
1 2
0 0
2 0
2 3
2 0
2 2
2 0
2 2
2 0
2 2
2 2
2 2
2 2
2 0
2 2
2 0
2 2
0 1
0 0
0 3
2 2
2 2
2 3
2 1
2 1
2 1
2 2
2 2
0 0
0 2
0 2
0 1
3 2
3 3
3 0
3 0
3 0
3 0
3 1
3 0
3 0
3 2
0 0
2 2
2 2
2 2
2 2
2 0
2 0
2 2
2 0
2 1
2 2
2 2
2 3
2 2
2 0
2 1
2 0
2 0
2 1
2 0
2 1
2 0
1 2
1 0
0 0
0 2
0 2
0 0
2 2
2 0
2 1
2 2
2 2
2 0
2 0
2 0
2 2
2 0
2 2
2 1
0 2
0 0
0 1
2 1
2 0
2 2
0 1
0 3
0 0
2 2
2 2
2 2
2 2
2 1
2 0
2 0
2 2
2 2
2 2
2 2
2 0
2 0
2 1
2 1
2 2
2 2
2 2
2 2
2 0
2 2
2 3
2 3
2 2
2 2
2 0
2 0
2 0
2 3
2 0
2 0
2 1
0 2
0 2
0 0
0 2
0 0
0 1
0 1
2 3
2 2
2 2
2 0
0 0
0 0
2 3
2 0
2 0
2 0
2 0
2 3
2 2
2 0
2 0
2 0
0 2
2 1
0 0
2 1
0 1
0 0
0 0
1 0
1 3
1 2
0 0
2 2
2 2
2 0
2 3
2 0
2 2
2 2
2 2
2 2
2 3
2 2
0 2
2 1
2 2
2 2
2 2
2 3
2 0
2 0
0 0
0 1
1 0
2 2
2 3
0 2
0 0
0 2
0 2
0 0
0 2
0 2
1 0
1 0
1 0
1 0
1 2
0 0
0 1
0 2
0 0
0 0
2 0
0 2
0 2
0 2
0 0
0 0
0 0
2 0
2 2
2 2
2 0
2 1
2 1
2 1
2 2
2 0
2 3
2 2
2 2
0 0
0 0
0 2
1 0
1 0
1 1
2 2
0 1
0 2


0 0
0 2
0 3
0 0
0 0
0 0
0 2
2 2
2 0
2 0
2 0
2 1
2 1
2 0
2 1
2 1
2 0
2 0
2 1
2 1
0 2
0 3
2 3
2 0
1 2
2 1
0 2
0 2
1 2
0 0
0 0
0 0
1 1
2 3
2 2
2 0
2 2
2 1
2 0
1 3
2 1
2 2
2 2
2 3
2 2
2 2
2 0
0 3
0 2
0 0
1 0
1 1
1 1
1 3
0 1
0 0
0 0
0 2
0 0
0 2
0 2
0 1
0 2
0 0
1 1
1 1
2 0
2 2
1 3
1 2
1 2
1 0
2 2
2 1
1 2
2 1
0 0
0 0
0 2
0 3
0 2
0 3
0 2
0 3
0 2
0 3
0 0
0 0
0 0
0 0
0 1
0 1
0 3
0 0
0 0
0 2
0 0
2 1
2 2
2 3
2 2
2 3
2 1
0 0
0 0
2 1
2 0
2 0
2 0
2 0
2 2
2 2
2 0
2 0
2 0
0 0
0 0
0 0
0 0
0 3
1 0
1 2
1 0
1 0
0 2
0 3
0 1
0 0
1 0
2 0
2 2
2 3
0 0
1 1
1 1
1 0
1 0
2 2
2 0
0 3
2 0
2 1
2 3
2 0
0 0
0 0
0 3
1 1
2 1
1 1
0 0
0 2
1 1
1 0
1 0
0 0
0 1
0 0
0 0
0 2
2 0
2 3
2 0
2 2
2 0
2 0
2 3
1 0
1 0
1 3
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 3
2 0
2 2
2 1
2 2
2 0
2 2
2 2
2 3
2 2
2 0
2 0
2 0
2 2
2 1
2 0
2 0
2 1
2 3
2 0
2 2
2 2
0 0
0 0
2 0
0 2
0 0
0 0
0 1
0 0
0 1
3 0
3 0
3 0
3 3
3 2
3 0
3 1
3 0
3 2
3 0
0 0
0 0
2 0
0 0
2 0
0 3
2 2
1 2
1 0
2 2
2 1
2 2
2 2
2 1
2 2
0 0
0 0
0 0
0 2
0 1
0 0
0 0
0 1
2 0
2 0
2 2
2 0
2 2
2 0
2 0
2 0
2 0
